<a href="https://colab.research.google.com/github/CRosero/aml-project/blob/master/train_step_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the dataset from drive.
( You can find the zipped folder [here](https://drive.google.com/file/d/1XsRmyQYHfgRFJCOueXpJ37yyOCrKHO-W/view?usp=sharing))

In [1]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/gdrive/')
data_path = "/content/data"

use_complete_dataset = True

if (use_complete_dataset == True) and (not os.path.isfile('/content/data.zip')):
  print("download entire dataset")
  !gdown --id 1A2dBwPlCyXHTqmG1LRvPfVm6K21jWwUI # 3-5 min
  !jar xf  "/content/data.zip"
elif (use_complete_dataset == False) and (not os.path.isfile('/content/data.zip')):
  # Load cropped dataset containing only 10 images
  print("download cropped dataset")
  !gdown --id 1gPcwDJsNpyqcjKu225hnIxkURpjUA08i   
  !jar xf  "/content/data.zip"
else:
  print("dataset already downloaded")

if not os.path.isdir('/content/data'):
  print("Dataset doesn't exist")

Mounted at /gdrive/
download cropped dataset
Downloading...
From: https://drive.google.com/uc?id=1gPcwDJsNpyqcjKu225hnIxkURpjUA08i
To: /content/data.zip
100% 73.4M/73.4M [00:01<00:00, 42.7MB/s]


# Cloning the repository from github

In [2]:
# Clone the entire repo.
repo_path = "/content/cloned-repo"
if not os.path.isdir(repo_path):
  !git clone -l -s https://github.com/CRosero/aml-project.git cloned-repo
  %cd cloned-repo
else:
  print("Repository already cloned")
%cd /content/cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 144 (delta 58), reused 3 (delta 1), pack-reused 49
Receiving objects: 100% (144/144), 6.29 MiB | 5.89 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/cloned-repo
/content/cloned-repo
dataset     eval.py  model	train.ipynb  train_step_2.ipynb  utils.py
eval.ipynb  loss.py  README.md	train.py     train_step_3.ipynb


# Importing the libraries

In [3]:
import argparse
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from model.build_BiSeNet import BiSeNet
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import numpy as np
from utils import poly_lr_scheduler
from utils import reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu
from loss import DiceLoss
import torch.cuda.amp as amp
import os
import os.path as osp
import random
import matplotlib.pyplot as plt
import collections
import torchvision
from torchvision.transforms import InterpolationMode
from torch.utils import data
from PIL import Image
import json
# Dataset class:
from dataset.cityscapesDataSet import cityscapesDataSet

In [4]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [5]:
## -- VALIDATION --

def val(args, model, dataloader):
    print('start val!')
    # label_info = get_label_info(csv_path)
    with torch.no_grad():
        model.eval()
        precision_record = []
        hist = np.zeros((args.num_classes, args.num_classes))
        for i, (data, label) in enumerate(dataloader):
            label = label.type(torch.LongTensor)
            
            if torch.cuda.is_available() and args.use_gpu:
                data = data.cuda()
                label = label.cuda()

            # get RGB predict image
            predict = model(data).squeeze()
            predict = reverse_one_hot(predict)
            predict = np.array(predict.cpu())

            # get RGB label image
            label = label.squeeze()
            label = np.array(label.cpu())

            # compute per pixel accuracy
            precision = compute_global_accuracy(predict, label)
            hist += fast_hist(label.flatten(), predict.flatten(), args.num_classes)

            # there is no need to transform the one-hot array to visual RGB array
            # predict = colour_code_segmentation(np.array(predict), label_info)
            # label = colour_code_segmentation(np.array(label), label_info)
            precision_record.append(precision)
        
        precision = np.mean(precision_record)
        # miou = np.mean(per_class_iu(hist))
        miou_list = per_class_iu(hist)[:-1]
        # miou_dict, miou = cal_miou(miou_list, csv_path)
        miou = np.mean(miou_list)
        print('precision per pixel for test: %.3f' % precision)
        print('mIoU for validation: %.3f' % miou)
        # miou_str = ''
        # for key in miou_dict:
        #     miou_str += '{}:{},\n'.format(key, miou_dict[key])
        # print('mIoU for each class:')
        # print(miou_str)
        return precision, miou

In [6]:
## -- TRAINING --

def train(args, model, epoch_start_i, optimizer, dataloader_train, dataloader_val, miou_init=0):
    writer = SummaryWriter(comment=''.format(args.optimizer, args.context_path))

    scaler = amp.GradScaler()

    if args.loss == 'dice':
        loss_func = DiceLoss()
    elif args.loss == 'crossentropy':
        loss_func = torch.nn.CrossEntropyLoss(ignore_index=255)
    
    max_miou = miou_init
    step = 0
    
    for epoch in range(epoch_start_i, args.num_epochs):
        lr = poly_lr_scheduler(optimizer, args.learning_rate, iter=epoch, max_iter=args.num_epochs)
        model.train()
        tq = tqdm(total=len(dataloader_train) * args.batch_size)
        tq.set_description('epoch %d, lr %f' % (epoch, lr))
        loss_record = []
        for i, (data, label) in enumerate(dataloader_train):
            if torch.cuda.is_available() and args.use_gpu:
                data = data.cuda()
                label = label.long().cuda()
  
            optimizer.zero_grad()
            
            with amp.autocast():
                output, output_sup1, output_sup2 = model(data)
                loss1 = loss_func(output, label)
                loss2 = loss_func(output_sup1, label)
                loss3 = loss_func(output_sup2, label)
                loss = loss1 + loss2 + loss3
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            tq.update(args.batch_size)
            #tq.set_postfix(loss='%.6f' % loss)
            step += 1
            writer.add_scalar('loss_step', loss, step)
            loss_record.append(loss.item())
            
        tq.close()
        loss_train_mean = np.mean(loss_record)
        writer.add_scalar('epoch/loss_epoch_train', float(loss_train_mean), epoch)
        print('loss for train : %f' % (loss_train_mean))
        
        if epoch % args.checkpoint_step == 0 and epoch != 0:
            print("Saving checkpoint")
            import os
            if not os.path.isdir(args.save_model_path):
                os.mkdir(args.save_model_path)
            torch.save({
                      'epoch': epoch,
                      'model_state_dict': model.module.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict()
                        },
                       os.path.join(args.save_model_path, 'latest_CE_loss.pth'))


        if epoch % args.validation_step == 0 and epoch != 0:
            precision, miou = val(args, model, dataloader_val)
            if miou > max_miou:
                max_miou = miou
                import os 
                os.makedirs(args.save_model_path, exist_ok=True)
                torch.save({
                      'miou': miou,
                      'epoch': epoch,
                      'model_state_dict': model.module.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict()
                        },
                       os.path.join(args.save_model_path, 'best_CE_loss.pth'))
            writer.add_scalar('epoch/precision_val', precision, epoch)
            writer.add_scalar('epoch/miou val', miou, epoch)

In [11]:
def main(params):
    # basic parameters
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_epochs', type=int, default=300, help='Number of epochs to train for')
    parser.add_argument('--epoch_start_i', type=int, default=0, help='Start counting epochs from this number')
    parser.add_argument('--checkpoint_step', type=int, default=10, help='How often to save checkpoints (epochs)')
    parser.add_argument('--validation_step', type=int, default=10, help='How often to perform validation (epochs)')
    parser.add_argument('--dataset', type=str, default="Cityscapes", help='Dataset you are using.')
    parser.add_argument('--crop_height', type=int, default=720, help='Height of cropped/resized input image to network')
    parser.add_argument('--crop_width', type=int, default=960, help='Width of cropped/resized input image to network')
    parser.add_argument('--batch_size', type=int, default=32, help='Number of images in each batch')
    parser.add_argument("--iter-size", type=int, default=1, help="Accumulate gradients for ITER_SIZE iterations.")
    parser.add_argument('--context_path', type=str, default="resnet101",
                        help='The context path model you are using, resnet18, resnet101.')
    parser.add_argument('--learning_rate', type=float, default=0.01, help='learning rate used for train')
    parser.add_argument('--data', type=str, default='content/data', help='path of training data')
    #parser.add_argument("--data-list", type=str, default='/gdrive/MyDrive/data/Cityscapes/train.txt', help="Path to the file listing the images in the source dataset.")
    #parser.add_argument("--data-list-val", type=str, default='/gdrive/MyDrive/data/Cityscapes/val.txt', help="Path to the file listing the image in the test subset.")
    parser.add_argument('--num_workers', type=int, default=4, help='num of workers')
    parser.add_argument('--num_classes', type=int, default=32, help='num of object classes (with void)')
    parser.add_argument("--num-steps", type=int, default=250000, help="Number of training steps.")
    parser.add_argument('--cuda', type=str, default='0', help='GPU ids used for training')
    parser.add_argument('--use_gpu', type=bool, default=True, help='whether to user gpu for training')
    parser.add_argument('--pretrained_model_path', type=str, default=None, help='path to pretrained model')
    parser.add_argument('--save_model_path', type=str, default=None, help='path to save model')
    parser.add_argument('--optimizer', type=str, default='rmsprop', help='optimizer, support rmsprop, sgd, adam')
    parser.add_argument('--loss', type=str, default='crossentropy', help='loss function, dice or crossentropy')
    parser.add_argument("--random-scale", action="store_true", help="Whether to randomly scale the inputs during the training.")
    parser.add_argument("--random-mirror", action="store_true", help="Whether to randomly mirror the inputs during the training.")
    parser.add_argument("--set-type", type=str, default='train', help="choose adaptation set.")
    parser.add_argument('--pretrained_best_model_path', type=str, default=None, help='path to the best pretrained model')


    args = parser.parse_args(params)

    # create dataset and dataloader
    data_root_path = os.path.join(args.data, args.dataset) # /content/data/Cityscapes
    train_path = os.path.join(data_root_path, "train.txt") # /content/data/Cityscapes/train.txt
    val_path = os.path.join(data_root_path, "val.txt")   # /content/data/Cityscapes/val.txt
    info_path = os.path.join(args.data, args.dataset, "info.json") # /content/data/Cityscapes/info.json 
    
    # preprocessing informations:
    input_size = (int(args.crop_height), int(args.crop_width))
    f = open(info_path)
    info = json.load(f)
    img_mean = info["mean"] # [73.15835921071155, 82.90891754262586, 72.39239876194159]  -> TODO: ask the tutor
    img_mean = np.array(img_mean, dtype=np.float32)
    #img_mean = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
    #img_mean=(128, 128, 128)

    
    # create dataloaders
    train_dataset = cityscapesDataSet(root=data_root_path,
                                      list_path = train_path,
                                      info_json = info,
                                      crop_size=input_size,
                                      scale=args.random_scale, 
                                      mirror=args.random_mirror, 
                                      mean=img_mean)
   
    
    val_dataset = cityscapesDataSet(root=data_root_path,
                                    list_path = val_path,
                                    info_json = info,
                                    crop_size=input_size,
                                    scale=False,
                                    mirror=args.random_mirror, 
                                    mean=img_mean)
    
    print(f'train_dataset: {len(train_dataset)}')
    print(f'val_dataset: {len(val_dataset)}')
    image, label = train_dataset[0]
    print(f'images shape: {image.shape}')
    print(f'label shape: {label.shape}')
    
    # Define dataloaders
    dataloader_train = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last = True, num_workers=args.num_workers, pin_memory=True)
    # (batch size for dataloader_val must be 1)
    dataloader_val = DataLoader(val_dataset, batch_size=1, shuffle=True, drop_last = True, num_workers=args.num_workers, pin_memory=True)
    
    # build model
    os.environ['CUDA_VISIBLE_DEVICES'] = args.cuda
    
    model = BiSeNet(args.num_classes, args.context_path)
    
    if torch.cuda.is_available() and args.use_gpu:
        model = torch.nn.DataParallel(model).cuda()

    # build optimizer
    if args.optimizer == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), args.learning_rate)
    elif args.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), args.learning_rate, momentum=0.9, weight_decay=1e-4)
    elif args.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), args.learning_rate)
    else:  # rmsprop
        print('not supported optimizer \n')
        return None

    epoch_start_i = 0
    miou_init = 0

    # load pretrained model if exists
    if (args.pretrained_model_path is not None) and (os.path.isfile(args.pretrained_model_path)):
        print('load model from %s ...' % args.pretrained_model_path)
        checkpoint= torch.load(args.pretrained_model_path)
        model.module.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch_start_i = int(checkpoint['epoch'])+1
        print('Done! Now starting from epoch:', epoch_start_i)

        if not os.path.isfile(args.pretrained_best_model_path):
            miou_init=0
        else:
          miou_init = torch.load(args.pretrained_best_model_path)['miou']
          print('getting best miou from lasting runs: ', miou_init)
        

    # train
    train(args, model, epoch_start_i, optimizer, dataloader_train, dataloader_val, miou_init)

    #val(args, model, dataloader_val)
    
    

In [8]:
%tensorboard --logdir=runs

<IPython.core.display.Javascript object>

In [13]:
if __name__ == '__main__':
    params = [
        '--num_epochs', '50',
        '--learning_rate', '2.5e-2',
        '--data', data_path,
        '--num_workers', '4',
        '--num_classes', '19',
        '--cuda', '0',
        '--batch_size', '16',  
        '--save_model_path', '/gdrive/MyDrive/Project_AML/Models/checkpoints_101_sgd/',
        '--pretrained_model_path', '/gdrive/MyDrive/Project_AML/Models/checkpoints_101_sgd/latest_CE_loss.pth',
        '--context_path', 'resnet101',  # set resnet18 or resnet101, only support resnet18 and resnet101
        '--optimizer', 'sgd',
        '--checkpoint_step', '2',        
        '--pretrained_best_model_path', '/gdrive/MyDrive/Project_AML/Models/checkpoints_101_sgd/best_CE_loss.pth',
        '--validation_step', '2'
    ]
    main(params)

train_dataset: 10
val_dataset: 10
images shape: (3, 960, 720)
label shape: (960, 720)


epoch 0, lr 0.025000: 100%|██████████| 9/9 [00:19<00:00,  2.14s/it]


loss for train : 8.332841


epoch 1, lr 0.024550: 100%|██████████| 9/9 [00:19<00:00,  2.12s/it]


loss for train : 5.850342


...

loss for train : 4.660455
Saving checkpoint
start val!
precision per pixel for test: 0.319
mIoU for validation: 0.067


epoch 3, lr 0.023646: 100%|██████████| 9/9 [00:19<00:00,  2.14s/it]


loss for train : 3.811675


epoch 4, lr 0.023193:   0%|          | 0/9 [00:00<?, ?it/s]Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: ignored

In [ ]:
torch.cuda.memory_summary()
torch.cuda.empty_cache()